In [108]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import datetime

In [116]:
year = 2019
cal = 2 # 1 for WT, 2 for UCI, etc.
results_dict = {}
stages_dict = {}

url = 'https://firstcycling.com/race.php?y='+str(year)+'&t='+str(cal)
r = requests.get(url)
soup = BeautifulSoup(r.text,'html.parser')

if year == datetime.datetime.today().year:
    tab = soup.find_all('table', class_='sortTabell tablesorter')[1]
else:
    tab = soup.find_all('table', class_='sortTabell tablesorter')[0]

refs = [i['href'] for i in tab.find_all('a', href=True)]
races_raw = [x for x in refs if 'race' in x]
races = [z for z in races_raw if 'k' not in z]
urls = ['https://firstcycling.com/'+y for y in races]
for race in urls:
    r_race = requests.get(race)
    soup_race = BeautifulSoup(r_race.text,'html.parser')
    race_name = soup_race.find('h1').text
    rows = soup_race.find_all('tbody')[0].find_all('tr')
    result = []
    for tr in rows:
        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td if tr.text.strip()]
        if row:
            result.append(row)
            
    stage_soup = BeautifulSoup(requests.get(race+'&k=2').text,'html.parser') 
    try:
        if stage_soup.find('h1', class_='delinfo').text == 'Stages':
            stg_ref = [i['href'] for i in stage_soup.find_all('table', class_="tablesorter")[1].find_all('a')]
            stages = [x for x in stg_ref if 'race' in x]
            stages_url = ['https://firstcycling.com/'+y for y in stages]
            for stage in stages_url:
                r_stage = requests.get(stage)
                soup_stage = BeautifulSoup(r_stage.text,'html.parser')
                for i in soup_stage.find_all('select')[1].find_all('option'):
                    if i.has_attr('selected'):
                        stage_name = race_name+' stage '+i.text
                rows_stage = soup_stage.find_all('tbody')[0].find_all('tr')
                result_stage = []
                for tr in rows_stage:
                    td = tr.find_all('td')
                    row = [tr.text.strip() for tr in td if tr.text.strip()]
                    if row:
                        result_stage.append(row)
                stages_dict[stage_name] = pd.DataFrame(result_stage,
                                                       columns=['Position', 'Rider', 'Born', 'Team', 'Total time', 'FC points'])
    except:
        continue
            
            
    try:        
        results_dict[race_name] = pd.DataFrame(result, 
                                               columns=['Position', 'Rider', 'Born', 'Team', 'Total time', 'FC points'])
    except ValueError:
        continue